In [16]:
import squigglepy as sq
import numpy as np
import pandas as pd
from squigglepy.numbers import K, M, B

sq.set_seed(42)
np.random.seed(42)
np.seterr(invalid='raise')  # Warn on operations involving NaN
N_SAMPLES = 5000

from chip_estimates_utils import (
    normalize_shares,
    estimate_chip_sales,
    estimate_cumulative_chip_sales,
    aggregate_by_chip_type,
    interpolate_samples_to_calendar_quarters,
    compute_running_totals,
)


In [17]:
# ==============================================
# GOOGLE SHEETS CONFIGURATION
# ==============================================

SPREADSHEET_ID = "1eGk2AAdewEO81vx-YBRTtdlhZvAMstY7vZuHrf3sgNI"

REVENUE_SHEET = "TPU_Revenue"
PROD_MIX_SHEET = "Production_Mix"
PRICES_SHEET = "prices"

# Construct URLs for direct CSV export
REVENUE_URL = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&sheet={REVENUE_SHEET}"
PROD_MIX_URL = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&sheet={PROD_MIX_SHEET}"
PRICES_URL = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&sheet={PRICES_SHEET}"


In [18]:
# ==============================================
# LOAD DATA FROM GOOGLE SHEETS
# ==============================================

revenue_df = pd.read_csv(REVENUE_URL).dropna(axis=1, how="all")
prod_mix_df = pd.read_csv(PROD_MIX_URL).dropna(axis=1, how="all")
prices_df = pd.read_csv(PRICES_URL).dropna(axis=1, how="all")

QUARTERS = revenue_df['quarter'].tolist()
TPU_VERSIONS = prod_mix_df['version'].dropna().unique().tolist()

print(f"Loaded {len(revenue_df)} quarters of revenue data")
print(revenue_df[['quarter', 'start_date', 'end_date', 'revenue_p5', 'revenue_p95', 'broadcom_margin_p5', 'broadcom_margin_p95']].head())
print()
print(prod_mix_df[['quarter', 'version', 'share_p5', 'share_p95']].head(10))
print()
print(prices_df)


Loaded 12 quarters of revenue data
   quarter  start_date    end_date  revenue_p5  revenue_p95  \
0  Q1_FY23  10/31/2022   1/29/2023        0.42         0.53   
1  Q2_FY23   1/30/2023   4/30/2023        0.53         0.66   
2  Q3_FY23    5/1/2023   7/30/2023        0.53         0.66   
3  Q4_FY23   7/31/2023  10/29/2023        0.79         0.99   
4  Q1_FY24  10/30/2023    2/4/2024        1.30         1.45   

   broadcom_margin_p5  broadcom_margin_p95  
0                0.55                 0.70  
1                0.55                 0.70  
2                0.55                 0.70  
3                0.55                 0.70  
4                0.50                 0.65  

   quarter version  share_p5  share_p95
0  Q1_FY23      v4      0.60       0.80
1  Q1_FY23     v4i      0.10       0.20
2  Q1_FY23     v5e      0.10       0.20
3  Q2_FY23      v4      0.60       0.90
4  Q2_FY23     v5e      0.10       0.40
5  Q3_FY23     v5e      0.65       0.95
6  Q3_FY23      v4      0.05       

In [19]:
# ==============================================
# CONVERT TO SQUIGGLEPY DISTRIBUTIONS
# ==============================================

TPU_REVENUE = {}
for _, row in revenue_df.iterrows():
    quarter = row['quarter']
    TPU_REVENUE[quarter] = sq.norm(row['revenue_p5'], row['revenue_p95'])

PROD_MIX = {}
for quarter in prod_mix_df['quarter'].unique():
    quarter_data = prod_mix_df[prod_mix_df['quarter'] == quarter]
    PROD_MIX[quarter] = {}
    for _, row in quarter_data.iterrows():
        version = row['version']
        PROD_MIX[quarter][version] = sq.norm(row['share_p5'], row['share_p95'], lclip=0, rclip=1)

# TPU manufacturing costs from prices sheet
TPU_COST = {row['version']: sq.to(row['cost_p5'], row['cost_p95']) for _, row in prices_df.iterrows()}

# Broadcom margins by quarter from revenue sheet
MARGIN_BY_QUARTER = {row['quarter']: sq.to(row['broadcom_margin_p5'], row['broadcom_margin_p95']) for _, row in revenue_df.iterrows()}

print("TPU costs (90% CI):")
for v, cost in TPU_COST.items():
    print(f"  {v}: ${cost.x:,.0f}-${cost.y:,.0f}")

print("Broadcom margins (first 6 quarters):")
for q in list(MARGIN_BY_QUARTER.keys())[:6]:
    m = MARGIN_BY_QUARTER[q]
    print(f"  {q}: {m.x:.0%}-{m.y:.0%}")

TPU costs (90% CI):
  v3: $940-$1,400
  v4i: $700-$1,100
  v4: $1,100-$1,500
  v5e: $950-$1,400
  v5p: $2,300-$2,900
  v6e: $1,600-$1,900
  v7: $4,600-$5,500
Broadcom margins (first 6 quarters):
  Q1_FY23: 55%-70%
  Q2_FY23: 55%-70%
  Q3_FY23: 55%-70%
  Q4_FY23: 55%-70%
  Q1_FY24: 50%-65%
  Q2_FY24: 50%-65%


In [20]:
# ==============================================
# CHIP SPECS FROM EPOCH.AI
# ==============================================
# Download chip specs including TOPS and TDP

import requests
import zipfile
import io

# Map notebook TPU version names to epoch.ai CSV names
TPU_NAME_MAP = {
    'v3': 'TPU v3',
    'v4i': 'TPU v4i', 
    'v4': 'TPU v4',
    'v5e': 'TPU v5e',
    'v5p': 'TPU v5p',
    'v6e': 'TPU v6e',
    'v7': 'TPU v7',
}

# Fallback specs (TOPS from 8-bit OP/s, TDP in watts)
FALLBACK_SPECS = {
    'v3':  {'tops': 123,  'tdp': 450},
    'v4i': {'tops': 138,  'tdp': 175},
    'v4':  {'tops': 275,  'tdp': 340},
    'v5e': {'tops': 393,  'tdp': 225},
    'v5p': {'tops': 918,  'tdp': 540},
    'v6e': {'tops': 1836, 'tdp': 380},
    'v7':  {'tops': 4614, 'tdp': 960},
}

H100_TOPS = 1979  # Reference for H100-equivalent calculation

def load_tpu_specs(chip_types_df, name_map, fallback_specs):
    """Extract TOPS and TDP from CSV, with fallbacks for missing values."""
    specs = {}
    for notebook_name, csv_name in name_map.items():
        row = chip_types_df[chip_types_df['Name'] == csv_name]
        if len(row) == 1:
            # Parse 8-bit OP/s
            tops_raw = row['8-bit OP/s'].values[0]
            tops = float(tops_raw) / 1e12 if pd.notna(tops_raw) else fallback_specs[notebook_name]['tops']
            
            # Parse TDP
            tdp_col = 'TDP (W) (from ML Hardware (linked))'
            tdp_raw = row[tdp_col].values[0] if tdp_col in row.columns else None
            tdp = float(tdp_raw) if pd.notna(tdp_raw) else fallback_specs[notebook_name]['tdp']
            
            specs[notebook_name] = {'tops': tops, 'tdp': tdp}
        else:
            specs[notebook_name] = fallback_specs[notebook_name].copy()
            print(f"Warning: Using fallback specs for {notebook_name} (not found in CSV)")
    return specs

# Try to download and parse chip specs from epoch.ai
try:
    url = "https://epoch.ai/data/ai_chip_sales.zip"
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        with z.open("chip_types.csv") as f:
            chip_types_df = pd.read_csv(f)
            tpu_chips_df = chip_types_df[chip_types_df['Designer'] == 'Google']
    
    print(tpu_chips_df[['Name', 'TDP (W) (from ML Hardware (linked))', '8-bit OP/s']].head(10))
    TPU_SPECS = load_tpu_specs(tpu_chips_df, TPU_NAME_MAP, FALLBACK_SPECS)
    print("\nLoaded TPU specs from epoch.ai CSV:")
except Exception as e:
    print(f"Warning: Could not download chip specs from epoch.ai: {e}")
    print("Using fallback TPU specs:")
    TPU_SPECS = FALLBACK_SPECS.copy()

for chip, spec in TPU_SPECS.items():
    print(f"  {chip}: TOPS={spec['tops']:.0f}, TDP={spec['tdp']:.0f}W")

       Name  TDP (W) (from ML Hardware (linked))        8-bit OP/s
0    TPU v3                                450.0   123000000000000
1   TPU v4i                                175.0   138000000000000
3    TPU v4                                340.0   275000000000000
11  TPU v5e                                225.0   393000000000000
13  TPU v5p                                540.0   918000000000000
16  TPU v6e                                380.0  1836000000000000
25   TPU v7                                960.0  4614000000000000

Loaded TPU specs from epoch.ai CSV:
  v3: TOPS=123, TDP=450W
  v4i: TOPS=138, TDP=175W
  v4: TOPS=275, TDP=340W
  v5e: TOPS=393, TDP=225W
  v5p: TOPS=918, TDP=540W
  v6e: TOPS=1836, TDP=380W
  v7: TOPS=4614, TDP=960W


In [21]:
# ==============================================
# SAMPLING FUNCTIONS
# ==============================================

# Base quarter for correlated price sampling (prices sampled once per chip, using this quarter's margin)
BASE_QUARTER = QUARTERS[0]


def sample_revenue(quarter):
    return (TPU_REVENUE[quarter] @ 1) * B


def sample_shares(quarter):
    mix = PROD_MIX[quarter]
    raw_shares = {version: dist @ 1 for version, dist in mix.items()}
    return normalize_shares(raw_shares)


def sample_base_price(version):
    """Sample base price = cost / (1 - margin) using base quarter margin."""
    return (TPU_COST[version] / (1 - MARGIN_BY_QUARTER[BASE_QUARTER])) @ 1


# Uncorrelated price sampler (for comparison)
def sample_price(quarter, version):
    """Sample price = cost / (1 - margin)"""
    return (TPU_COST[version] / (1 - MARGIN_BY_QUARTER[quarter])) @ 1


# Margin deflation factor for correlated model
# 
# We want to adjust prices based on each quarter's margin while keeping price
# uncertainty correlated across time. The base price is sampled once using
# BASE_QUARTER's margin, then scaled by this deflation factor for other quarters.
#
# We compute this empirically rather than using closed-form because:
# 1. price = cost / (1 - margin), and 1/(1-x) is convex (Jensen's inequality)
# 2. We want E[1/(1-margin_q)] / E[1/(1-margin_base)], not the ratio of medians
# 3. For lognormal margins, the mean of the transform ≠ transform of the mean
#
# Since margin only changes once (FY23: 55-70% → FY24+: 50-65%), we compute
# the deflation factor once and apply it based on fiscal year.

def _compute_expected_price_multiplier(margin_dist, n=10000):
    """Compute E[1/(1-margin)] by sampling."""
    samples = margin_dist @ n
    return np.mean(1 / (1 - samples))

# Compute deflation factor for FY24+ relative to FY23
FY23_MARGIN = sq.to(0.55, 0.70)
FY24_MARGIN = sq.to(0.50, 0.65)
FY24_DEFLATION = _compute_expected_price_multiplier(FY24_MARGIN) / _compute_expected_price_multiplier(FY23_MARGIN)

def get_margin_deflation_factor(quarter, version):
    """Return price adjustment factor: 1.0 for FY23, ~0.88 for FY24+."""
    if 'FY23' in quarter:
        return 1.0
    return FY24_DEFLATION

print(f"FY24+ deflation factor: {FY24_DEFLATION:.4f}")

FY24+ deflation factor: 0.8763


In [22]:
# ==============================================
# RUN CORRELATED SIMULATION
# ==============================================

quarterly_samples = estimate_cumulative_chip_sales(
    quarters=QUARTERS,
    chip_types=TPU_VERSIONS,
    sample_revenue=sample_revenue,
    sample_shares=sample_shares,
    sample_base_price=sample_base_price,
    get_deflation_factor=get_margin_deflation_factor,
    sample_revenue_uncertainty=None,
    n_samples=N_SAMPLES,
)

cumulative_samples = aggregate_by_chip_type(quarterly_samples)

print("Simulation complete.")

Simulation complete.


In [23]:
# ==============================================
# CUMULATIVE SUMMARY
# ==============================================

def print_cumulative_summary(cumulative_samples, versions, title="Cumulative Production"):
    print(f"{title}")
    print(f"{'Version':<8} {'p5':>12} {'p50':>12} {'p95':>12}")
    print("-" * 51)

    grand_total = None
    for v in versions:
        arr = cumulative_samples[v]
        if arr.sum() > 0:
            if grand_total is None:
                grand_total = np.zeros_like(arr)
            grand_total += arr
            print(f"{v:<8} {int(np.percentile(arr, 5)):>12,} {int(np.percentile(arr, 50)):>12,} {int(np.percentile(arr, 95)):>12,}")

    if grand_total is not None:
        print("-" * 51)
        print(f"{'TOTAL':<8} {int(np.percentile(grand_total, 5)):>12,} {int(np.percentile(grand_total, 50)):>12,} {int(np.percentile(grand_total, 95)):>12,}")

print_cumulative_summary(cumulative_samples, TPU_VERSIONS, "Cumulative TPU Production (Correlated Model)")

Cumulative TPU Production (Correlated Model)
Version            p5          p50          p95
---------------------------------------------------
v4            192,512      262,025      344,458
v4i            18,098       30,092       45,637
v5e         1,762,157    2,380,572    3,156,760
v5p           498,889      670,697      871,666
v6e         1,765,811    2,294,854    2,839,507
v7             37,314       81,427      123,946
---------------------------------------------------
TOTAL       4,926,838    5,736,957    6,675,640


In [24]:
# ==============================================
# CUMULATIVE RUNNING TOTALS BY FISCAL QUARTER
# ==============================================

running_totals_samples = compute_running_totals(quarterly_samples)

print("Cumulative Running Totals by Fiscal Quarter")
print(f"{'Quarter':<10} {'Version':<8} {'p5':>12} {'p50':>12} {'p95':>12}")
print("=" * 60)

for quarter in QUARTERS:
    quarter_has_data = False
    for v in TPU_VERSIONS:
        arr = running_totals_samples[quarter][v]
        if arr.sum() > 0:
            quarter_has_data = True
            print(f"{quarter:<10} {v:<8} {int(np.percentile(arr, 5)):>12,} {int(np.percentile(arr, 50)):>12,} {int(np.percentile(arr, 95)):>12,}")

    if quarter_has_data:
        total = sum(running_totals_samples[quarter][v] for v in TPU_VERSIONS)
        print(f"{quarter:<10} {'TOTAL':<8} {int(np.percentile(total, 5)):>12,} {int(np.percentile(total, 50)):>12,} {int(np.percentile(total, 95)):>12,}")
        print("-" * 60)

Cumulative Running Totals by Fiscal Quarter
Quarter    Version            p5          p50          p95
Q1_FY23    v4             71,348       97,163      128,584
Q1_FY23    v4i            18,098       30,092       45,637
Q1_FY23    v5e            14,316       23,118       34,661
Q1_FY23    TOTAL         121,663      151,706      185,436
------------------------------------------------------------
Q2_FY23    v4            168,924      228,289      298,463
Q2_FY23    v4i            18,098       30,092       45,637
Q2_FY23    v5e            41,227       70,631      106,766
Q2_FY23    TOTAL         268,213      331,189      401,977
------------------------------------------------------------
Q3_FY23    v4            192,512      262,025      344,458
Q3_FY23    v4i            18,098       30,092       45,637
Q3_FY23    v5e           161,974      227,633      314,162
Q3_FY23    TOTAL         429,572      522,967      629,893
------------------------------------------------------------
Q4_FY2

In [25]:
# ==============================================
# CALENDAR QUARTER INTERPOLATION (SAMPLE-BASED)
# ==============================================

quarter_dates = {q: (revenue_df.loc[revenue_df['quarter'] == q, 'start_date'].iloc[0],
                     revenue_df.loc[revenue_df['quarter'] == q, 'end_date'].iloc[0])
                 for q in QUARTERS}

calendar_quarterly_samples = interpolate_samples_to_calendar_quarters(quarterly_samples, quarter_dates)
calendar_running_totals_samples = compute_running_totals(calendar_quarterly_samples)

print("Cumulative Running Totals by Calendar Quarter")
print(f"{'Quarter':<10} {'Version':<8} {'p5':>12} {'p50':>12} {'p95':>12}")
print("=" * 60)

for cq in calendar_running_totals_samples:
    quarter_has_data = False
    for v in TPU_VERSIONS:
        arr = calendar_running_totals_samples[cq][v]
        if arr.sum() > 0:
            quarter_has_data = True
            print(f"{cq:<10} {v:<8} {int(np.percentile(arr, 5)):>12,} {int(np.percentile(arr, 50)):>12,} {int(np.percentile(arr, 95)):>12,}")

    if quarter_has_data:
        total = sum(calendar_running_totals_samples[cq][v] for v in TPU_VERSIONS)
        print(f"{cq:<10} {'TOTAL':<8} {int(np.percentile(total, 5)):>12,} {int(np.percentile(total, 50)):>12,} {int(np.percentile(total, 95)):>12,}")
        print("-" * 60)

Cumulative Running Totals by Calendar Quarter
Quarter    Version            p5          p50          p95
Q4 2022    v4             48,611       66,199       87,606
Q4 2022    v4i            12,330       20,502       31,093
Q4 2022    v5e             9,754       15,751       23,615
Q4 2022    TOTAL          82,891      103,360      126,341
------------------------------------------------------------
Q1 2023    v4            136,993      185,119      241,643
Q1 2023    v4i            18,098       30,092       45,637
Q1 2023    v5e            33,798       55,082       81,712
Q1 2023    TOTAL         221,067      272,061      329,984
------------------------------------------------------------
Q2 2023    v4            185,378      250,959      328,629
Q2 2023    v4i            18,098       30,092       45,637
Q2 2023    v5e           123,906      176,092      244,450
Q2 2023    TOTAL         378,264      460,140      553,250
------------------------------------------------------------
Q3 2

In [26]:
# ==============================================
# CSV EXPORTS: CUMULATIVE RUNNING TOTALS BY CHIP TYPE
# ==============================================
# Export calendar quarter running totals broken down by TPU version

def get_calendar_quarter_dates(cal_q):
    """Return (start_date, end_date) strings for a calendar quarter like 'Q1 2024'."""
    parts = cal_q.split()
    q_num = int(parts[0][1])
    year = int(parts[1])
    if q_num == 1:
        return f"1/1/{year}", f"3/31/{year}"
    elif q_num == 2:
        return f"4/1/{year}", f"6/30/{year}"
    elif q_num == 3:
        return f"7/1/{year}", f"9/30/{year}"
    else:
        return f"10/1/{year}", f"12/31/{year}"

# Get first calendar quarter dynamically
first_calendar_quarter = list(calendar_running_totals_samples.keys())[0]
first_start_date, _ = get_calendar_quarter_dates(first_calendar_quarter)

rows = []
for cq in calendar_running_totals_samples:
    _, end_date = get_calendar_quarter_dates(cq)
    for version in TPU_VERSIONS:
        arr = calendar_running_totals_samples[cq][version]
        if arr.sum() > 0:
            display_name = f"TPU {version}"
            rows.append({
                'Name': f"{display_name} {first_calendar_quarter} to {cq}",
                'Chip manufacturer': 'Google',
                'Start date': first_start_date,
                'End date': end_date,
                'Chip type': display_name,
                'Number of units (5th percentile)': int(np.percentile(arr, 5)),
                'Number of units (median)': int(np.percentile(arr, 50)),
                'Number of units (95th percentile)': int(np.percentile(arr, 95)),
            })

by_chip_df = pd.DataFrame(rows)
by_chip_df.to_csv('tpu_cumulative_by_chip.csv', index=False)
print(f"Exported {len(by_chip_df)} rows to tpu_cumulative_by_chip.csv")
# print(by_chip_df.head(10))

Exported 56 rows to tpu_cumulative_by_chip.csv


In [27]:
# ==============================================
# CSV EXPORTS: FULL-TPU AGGREGATE STATS
# ==============================================
# Export calendar quarter running totals with aggregate metrics across all TPU versions
# Metrics: total units, H100e compute, power (MW)

# Get first calendar quarter dynamically
first_calendar_quarter = list(calendar_running_totals_samples.keys())[0]
first_start_date, _ = get_calendar_quarter_dates(first_calendar_quarter)

rows = []
for cq in calendar_running_totals_samples:
    _, end_date = get_calendar_quarter_dates(cq)
    
    # Compute aggregate metrics across all TPU versions
    n_samples = N_SAMPLES
    units_total = np.zeros(n_samples)
    h100e_total = np.zeros(n_samples)
    power_total_w = np.zeros(n_samples)
    
    for version in TPU_VERSIONS:
        arr = calendar_running_totals_samples[cq][version]
        units_total += arr
        if version in TPU_SPECS:
            tops = TPU_SPECS[version]['tops']
            tdp = TPU_SPECS[version]['tdp']
            h100e_total += arr * (tops / H100_TOPS)
            power_total_w += arr * tdp
    
    rows.append({
        'Name': f"Google TPU total {first_calendar_quarter} to {cq}",
        'Chip manufacturer': 'Google',
        'Designer': 'Google',
        'Start date': first_start_date,
        'End date': end_date,
        'Number of units (5th percentile)': int(np.percentile(units_total, 5)),
        'Number of units (median)': int(np.percentile(units_total, 50)),
        'Number of units (95th percentile)': int(np.percentile(units_total, 95)),
        'Compute estimate in H100e (5th percentile)': int(np.percentile(h100e_total, 5)),
        'Compute estimate in H100e (median)': int(np.percentile(h100e_total, 50)),
        'Compute estimate in H100e (95th percentile)': int(np.percentile(h100e_total, 95)),
        'Power in MW (5th percentile)': int(np.percentile(power_total_w / 1e6, 5)),
        'Power in MW (median)': int(np.percentile(power_total_w / 1e6, 50)),
        'Power in MW (95th percentile)': int(np.percentile(power_total_w / 1e6, 95)),
    })

totals_df = pd.DataFrame(rows)
totals_df.to_csv('tpu_cumulative_totals.csv', index=False)
print(f"Exported {len(totals_df)} rows to tpu_cumulative_totals.csv")
# print(totals_df)

Exported 13 rows to tpu_cumulative_totals.csv


In [28]:
# ==============================================
# CHRONOLOGICAL VIEW: FISCAL + CALENDAR INTERLEAVED
# ==============================================

from datetime import datetime

selected_fiscal = QUARTERS[:5]
selected_calendar = list(calendar_running_totals_samples.keys())[:4]

timeline = []

for q in selected_fiscal:
    end_date = pd.to_datetime(revenue_df.loc[revenue_df['quarter'] == q, 'end_date'].iloc[0])
    timeline.append({
        'end_date': end_date,
        'label': q,
        'data': running_totals_samples[q],
        'type': 'FISCAL',
    })

for cq in selected_calendar:
    parts = cq.split()
    q_num, year = int(parts[0][1]), int(parts[1])
    end_dates = {1: (3, 31), 2: (6, 30), 3: (9, 30), 4: (12, 31)}
    end_date = datetime(year, *end_dates[q_num])
    timeline.append({
        'end_date': end_date,
        'label': cq,
        'data': calendar_running_totals_samples[cq],
        'type': 'CALENDAR',
    })

# Sort chronologically
timeline.sort(key=lambda x: x['end_date'])

print("Chronological Comparison: Fiscal vs Calendar Quarter Running Totals")
print("=" * 80)

for entry in timeline:
    print(f"\n{entry['type']}: {entry['label']} (ends {entry['end_date'].strftime('%Y-%m-%d')})")
    print(f"  {'Version':<8} {'p5':>12} {'p50':>12} {'p95':>12}")
    print(f"  {'-'*50}")

    total = np.zeros(N_SAMPLES)
    for v in TPU_VERSIONS:
        arr = entry['data'][v]
        if arr.sum() > 0:
            total += arr
            print(f"  {v:<8} {int(np.percentile(arr, 5)):>12,} {int(np.percentile(arr, 50)):>12,} {int(np.percentile(arr, 95)):>12,}")

    print(f"  {'-'*50}")
    print(f"  {'TOTAL':<8} {int(np.percentile(total, 5)):>12,} {int(np.percentile(total, 50)):>12,} {int(np.percentile(total, 95)):>12,}")

Chronological Comparison: Fiscal vs Calendar Quarter Running Totals

CALENDAR: Q4 2022 (ends 2022-12-31)
  Version            p5          p50          p95
  --------------------------------------------------
  v4             48,611       66,199       87,606
  v4i            12,330       20,502       31,093
  v5e             9,754       15,751       23,615
  --------------------------------------------------
  TOTAL          82,891      103,360      126,341

FISCAL: Q1_FY23 (ends 2023-01-29)
  Version            p5          p50          p95
  --------------------------------------------------
  v4             71,348       97,163      128,584
  v4i            18,098       30,092       45,637
  v5e            14,316       23,118       34,661
  --------------------------------------------------
  TOTAL         121,663      151,706      185,436

CALENDAR: Q1 2023 (ends 2023-03-31)
  Version            p5          p50          p95
  --------------------------------------------------
  v4    

In [29]:
# ==============================================
# UNCORRELATED MODEL COMPARISON
# ==============================================

uncorrelated_samples = estimate_chip_sales(
    quarters=QUARTERS,
    versions=TPU_VERSIONS,
    sample_revenue=sample_revenue,
    sample_shares=sample_shares,
    sample_price=sample_price,
    n_samples=1000,  # smaller for speed
)

cumulative_uncorrelated_samples = aggregate_by_chip_type(uncorrelated_samples)

In [30]:
print_cumulative_summary(cumulative_uncorrelated_samples, TPU_VERSIONS, "Cumulative TPU Production (Uncorrelated Model)")
print(" ")
print_cumulative_summary(cumulative_samples, TPU_VERSIONS, "Cumulative TPU Production (Correlated Model)")

Cumulative TPU Production (Uncorrelated Model)
Version            p5          p50          p95
---------------------------------------------------
v4            210,268      262,021      323,834
v4i            18,181       30,295       47,363
v5e         2,094,560    2,390,734    2,725,625
v5p           546,282      667,414      799,985
v6e         1,998,243    2,271,279    2,577,914
v7             34,736       78,886      121,101
---------------------------------------------------
TOTAL       5,307,107    5,717,692    6,131,752
 
Cumulative TPU Production (Correlated Model)
Version            p5          p50          p95
---------------------------------------------------
v4            192,512      262,025      344,458
v4i            18,098       30,092       45,637
v5e         1,762,157    2,380,572    3,156,760
v5p           498,889      670,697      871,666
v6e         1,765,811    2,294,854    2,839,507
v7             37,314       81,427      123,946
------------------------------